# Modelo de recomendacion

### Cargo el dataset usado para el modelo

In [1]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split as train_test_splitSV
from sklearn.preprocessing import LabelEncoder
import pickle
import pandas as pd
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise import accuracy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler


In [ ]:
user_reviews = pd.read_csv('../datasets/user_reviews.csv',usecols=['user_id','item_id','sentiment_analysis','recommend'])
label_encoder = LabelEncoder()
#elimino la columna user_id
user_reviews['user_id_num'] = label_encoder.fit_transform(user_reviews['user_id'])



### Voy a generar un rating a partir de recommend y sentiment_analysis para que este entre 0 y 5

In [ ]:
user_reviews['rating'] = np.where(user_reviews['recommend'] == True,  # Si 'recommend' es True
                                 np.where(user_reviews['sentiment_analysis'] == 2, 5,  # Si 'sentimiento' es positivo
                                          np.where(user_reviews['sentiment_analysis'] == 1, 3,  # Si 'sentimiento' es neutro
                                                   1)),  # Si 'sentimiento' es negativo cuando 'recommend' es True
                                 np.where(user_reviews['sentiment_analysis'] == 2, 4,  # Si 'sentimiento' es positivo
                                          np.where(user_reviews['sentiment_analysis'] == 1, 2,  # Si 'sentimiento' es neutro
                                                   0)))  # Si 'sentimiento' es negativo cuando 'recommend' es False

In [ ]:
user_reviews.sample(5)

,user_id,item_id,recommend,sentiment_analysis,user_id_num,rating
50605,76561198063055125,239070,True,2,5348,5
53520,76561198076114293,17460,True,2,7647,5
46727,newhollandarmy,223470,True,1,22668,3
46078,76561198024028497,227100,False,0,1647,0
53714,TheMarshmallowMan101,730,True,2,17361,5


In [ ]:
print(user_reviews['rating'].max())
print(user_reviews['rating'].min())

5
0


### Genero id numericos para user_id usando label encoder

In [ ]:
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(user_reviews[['user_id_num','item_id','rating']], reader)


### Separo el dataset en entrenamiento y testeo

In [ ]:
train,test = train_test_splitSV(data,test_size = .25)

## Voy a usar un modelo De descomposicion en valor singular (SVD), el cual es un filtro colaborativo
### Voy a usar los datos de 
* user_id
* item_id
* sentiment_analysis: como rating

### Aplico GridSearch para encontrar al modelo con los mejores hiperparametros

In [ ]:
param_grid = {'n_factors': [5,50,100],'n_epochs': [5, 10,20], 'lr_all': [0.001, 0.002, 0.005],
              'reg_all': [0.002, 0.02, 0.2]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs = -1)
gs.fit(data)


In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
best_model = gs.best_estimator['rmse']

1.5840100111706839
{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}


### Veo que el modelo tiene un rmse de 1.06 lo cual esta cerca de 1lo cual en la escala esta bien
### Ademas veo los mejores hiperparametros

## Por ultimo defino el modelo con los mejores hiperparametros y lo exporto

In [ ]:
model = SVD(n_factors=100,n_epochs=20,lr_all=0.005,reg_all=0.2)
model.fit(train)

In [ ]:
from surprise import accuracy
train_predictions = model.test(train.build_testset())
test_predictions = model.test(test)
print(f"RMSE en el conjunto de entrenamiento: {accuracy.rmse(train_predictions)}")
print(f"RMSE en el conjunto de prueba: {accuracy.rmse(test_predictions)}")

RMSE: 1.2764
RMSE en el conjunto de entrenamiento: 1.2764039278981176
RMSE: 1.5817
RMSE en el conjunto de prueba: 1.5817043122021668


### EL modelo obtuvo un mejor rendimiento en los datos de testeo el rmse 1.5 me indica que en en el ranking de 0 5 lo que indica una desviacion de 1.5 si bien tiene mucho rango de mejora para las columnas usadas como rating esta bastante bien

In [ ]:
with open('./SVD_model.pkl', 'wb') as file: # Exporto mi modelo
    pickle.dump(model, file)

In [ ]:
print(model.predict(4124,12354))
print(model.predict(4124,1244))
print(model.predict(4124,154))
print(model.predict(4124,12554))
print(model.predict(4124,1213))
print(model.predict(4124,1884))
print(model.predict(4124,35354))
print(model.predict(4124,1324))
print(model.predict(4124,13224))


user: 4124       item: 12354      r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1244       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 154        r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 12554      r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1213       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1884       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 35354      r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 1324       r_ui = None   est = 3.82   {'was_impossible': False}
user: 4124       item: 13224      r_ui = None   est = 3.82   {'was_impossible': False}


## Filtro basado en contenido con Vecinos Cercanos

In [2]:
# Este script trabaja con un dataset de juegos de Steam. Primero, cargamos el dataset utilizando pandas.
steam_games = pd.read_csv('../datasets/steam_games.csv')

# Luego identificamos y guardamos los nombres de las columnas del dataset que representan los géneros de los juegos.
generos = list(steam_games.drop(columns=['app_name','price','id','developer','Accounting','Year']).columns)

# Iniciamos una lista vacía para almacenar los perfiles de los items (juegos).
perfiles_items = []

# Iteramos sobre cada juego (fila) en el dataset y creamos una lista con los valores en cada una de las columnas de género.
for _, row in steam_games.iterrows():
    perfil_item = []
    for genero in generos:
        perfil_item.append(row[genero])
    # Añadimos la lista del perfil del item a la lista principal de perfiles.
    perfiles_items.append(perfil_item)

# Convertimos la lista de perfiles de items en un DataFrame de pandas, utilizando los nombres de géneros como nombres de columnas.
perfiles_items_df = pd.DataFrame(perfiles_items, columns=generos)

# Añadimos las columnas 'app_name' e 'id' desde el dataset original al nuevo DataFrame.
perfiles_items_df['app_name'] = steam_games['app_name']
perfiles_items_df['id'] = steam_games['id']

# Creamos un nuevo DataFrame que excluye la columna 'app_name', que servirá como nuestro modelo de datos.
data_model = perfiles_items_df.drop(['app_name'], axis=1)

# Utilizamos el StandardScaler para normalizar los valores en nuestro modelo de datos.
scaler = StandardScaler()
data_standar = scaler.fit_transform(data_model)

# Guardamos nuestros perfiles de items normalizados como un nuevo archivo csv.
perfiles_items_df.to_csv('../datasets/data_standar_nearest_model.csv',index=False)

# Definimos el valor de k para nuestro modelo k-nearest neighbors.
k = 5

# Creamos e entrenamos nuestro modelo k-nearest neighbors utilizando la métrica euclideana.
knn_model = NearestNeighbors(n_neighbors=k, metric='euclidean')
knn_model.fit(data_standar)

# Finalmente, guardamos nuestro modelo entrenado en un archivo .pkl para uso futuro.
with open('./NearestNeighnors.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

In [2]:
def item_recommend_func(item_id):
    # Cargar datos relevantes desde un archivo CSV.
    perfiles_items_df = pd.read_csv('../datasets/data_standar_nearest_model.csv')
    
    if item_id not in list(perfiles_items_df['id']):
        return {'Ese id no pertenece a un item registrado'}
    else:
        item_game = perfiles_items_df[perfiles_items_df['id'] == item_id]['app_name']

    # Eliminar la columna 'app_name' de los datos.
    data_model = perfiles_items_df.drop(['app_name'], axis=1)

    # Inicializar y ajustar el escalador de características.
    scaler = StandardScaler()
    data_standar = scaler.fit_transform(data_model)

    # Importamos modelo preentrenado de KNN
    with open('./NearestNeighnors.pkl', 'rb') as file:  
        knn_model = pickle.load(file)

    # Obtener el índice del artículo basado en su nombre.
    
    item_index = perfiles_items_df[perfiles_items_df['app_name'] == item_name].index[0]

    # Recuperar los vecinos más cercanos al artículo especificado.
    distances, indices = knn_model.kneighbors([data_standar[item_index]])

    # Crear un diccionario para almacenar las recomendaciones.
    recomendaciones = {}
    for i in range(1, len(indices[0])):  # Ignorar el primer vecino (ya que es el artículo mismo).
        # Identificar el índice y el nombre del artículo recomendado.
        recommended_item_index = indices[0][i]
        recommended_item_name = perfiles_items_df.loc[recommended_item_index, 'app_name']
        # Agregar recomendación al diccionario.
        recomendaciones[f"Recomendacion {i}"] = recommended_item_name

    # Devolver las recomendaciones como resultado.
    return recomendaciones

{'Recomendacion 1': "Deus Ex: Human Revolution - Director's Cut",
 'Recomendacion 2': 'NARUTO SHIPPUDEN: Ultimate Ninja STORM 3 Full Burst HD',
 'Recomendacion 3': 'System Shock 2',
 'Recomendacion 4': 'Magicka 2'}